# Chapter 36: Test Patterns and Organization

This notebook covers common testing patterns that make tests more readable, maintainable, and reliable. These patterns are framework-agnostic and apply to any Python testing approach.

## Key Concepts
- **AAA Pattern**: Arrange-Act-Assert structures tests into three clear phases
- **Test Doubles**: Stubs, spies, and fakes replace real dependencies in tests
- **Builder Pattern**: Fluent interface for constructing complex test data
- **Test Organization**: Naming conventions, grouping, and helper functions

## Section 1: The AAA Pattern (Arrange-Act-Assert)

The AAA pattern divides each test into three distinct phases:

1. **Arrange**: Set up the test data and preconditions
2. **Act**: Execute the code being tested
3. **Assert**: Verify the results

This structure makes tests easy to read and understand at a glance.

In [ ]:
# Example 1: A clean AAA test
def test_sort_preserves_all_elements() -> None:
    # Arrange
    items: list[int] = [3, 1, 4, 1, 5]

    # Act
    result: list[int] = sorted(items)

    # Assert
    assert result == [1, 1, 3, 4, 5]
    assert len(result) == len(items)
    print(f"Input:  {items}")
    print(f"Sorted: {result}")
    print("Test PASSED: sorted list has correct order and length")


test_sort_preserves_all_elements()

In [ ]:
# Example 2: AAA with more complex arrangement
class ShoppingCart:
    """A simple shopping cart."""

    def __init__(self) -> None:
        self._items: list[dict[str, str | float | int]] = []

    def add_item(self, name: str, price: float, quantity: int = 1) -> None:
        self._items.append({"name": name, "price": price, "quantity": quantity})

    def total(self) -> float:
        return sum(item["price"] * item["quantity"] for item in self._items)  # type: ignore[operator]

    def item_count(self) -> int:
        return sum(item["quantity"] for item in self._items)  # type: ignore[arg-type]


def test_cart_total_with_multiple_items() -> None:
    # Arrange
    cart: ShoppingCart = ShoppingCart()
    cart.add_item("Widget", price=9.99, quantity=2)
    cart.add_item("Gadget", price=24.99, quantity=1)

    # Act
    total: float = cart.total()
    count: int = cart.item_count()

    # Assert
    expected_total: float = 9.99 * 2 + 24.99
    assert abs(total - expected_total) < 0.01
    assert count == 3
    print(f"Total: ${total:.2f} (expected ${expected_total:.2f})")
    print(f"Item count: {count}")
    print("Test PASSED")


test_cart_total_with_multiple_items()

In [ ]:
# Example 3: AAA with exception testing
def divide(a: float, b: float) -> float:
    """Divide a by b, raising ValueError for zero divisor."""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b


def test_divide_by_zero_raises() -> None:
    # Arrange
    numerator: float = 10.0
    divisor: float = 0.0

    # Act & Assert (combined for exception testing)
    try:
        divide(numerator, divisor)
        assert False, "Expected ValueError"
    except ValueError as e:
        assert "Cannot divide by zero" in str(e)
        print(f"Caught expected error: {e}")
        print("Test PASSED")


test_divide_by_zero_raises()

## Section 2: AAA Anti-Patterns

Understanding common mistakes helps write better tests. Here are patterns to avoid.

In [ ]:
# Anti-pattern 1: Multiple Acts in one test (test does too much)
# BAD -- this tests two behaviors at once
def test_bad_multiple_acts() -> None:
    cart: ShoppingCart = ShoppingCart()

    cart.add_item("A", price=10.0)  # Act 1
    assert cart.total() == 10.0     # Assert 1

    cart.add_item("B", price=20.0)  # Act 2
    assert cart.total() == 30.0     # Assert 2

    # If Assert 1 fails, we never test Act 2


# GOOD -- separate tests, each with one act
def test_good_single_item_total() -> None:
    # Arrange
    cart: ShoppingCart = ShoppingCart()
    cart.add_item("A", price=10.0)
    # Act
    total: float = cart.total()
    # Assert
    assert total == 10.0


def test_good_two_item_total() -> None:
    # Arrange
    cart: ShoppingCart = ShoppingCart()
    cart.add_item("A", price=10.0)
    cart.add_item("B", price=20.0)
    # Act
    total: float = cart.total()
    # Assert
    assert total == 30.0


test_good_single_item_total()
test_good_two_item_total()
print("Both focused tests PASSED")
print("Tip: Each test should have exactly one Act phase")

In [ ]:
# Anti-pattern 2: Asserting on implementation details rather than behavior

class UserRepository:
    """Stores users in an internal list."""

    def __init__(self) -> None:
        self._users: list[dict[str, str]] = []

    def add(self, name: str, email: str) -> None:
        self._users.append({"name": name, "email": email})

    def find_by_name(self, name: str) -> dict[str, str] | None:
        for user in self._users:
            if user["name"] == name:
                return user
        return None

    def count(self) -> int:
        return len(self._users)


# BAD: Testing internal state
# assert repo._users == [{"name": "Alice", "email": "alice@test.com"}]

# GOOD: Testing through the public interface
def test_add_and_find_user() -> None:
    # Arrange
    repo: UserRepository = UserRepository()

    # Act
    repo.add("Alice", "alice@test.com")
    result: dict[str, str] | None = repo.find_by_name("Alice")

    # Assert (through public interface)
    assert result is not None
    assert result["email"] == "alice@test.com"
    assert repo.count() == 1
    print(f"Found user: {result}")
    print("Test PASSED: asserted through public interface, not internal state")


test_add_and_find_user()

## Section 3: Test Doubles -- Stubs

A **stub** is a test double that provides canned responses to calls. It replaces a real dependency with a simplified version that returns predetermined data. Stubs answer the question: *"Given this dependency returns X, what does my code do?"*

In [ ]:
from typing import Protocol


# Define the interface using a Protocol
class Database(Protocol):
    def query(self, sql: str) -> list[dict[str, str | int]]: ...
    def execute(self, sql: str) -> int: ...


# Stub implementation returns canned data
class DatabaseStub:
    """A stub that returns predetermined query results."""

    def query(self, sql: str) -> list[dict[str, str | int]]:
        return [{"id": 1, "name": "Alice"}, {"id": 2, "name": "Bob"}]

    def execute(self, sql: str) -> int:
        return 1  # Always reports 1 row affected


# Code under test
class UserService:
    """Service that depends on a database."""

    def __init__(self, db: Database) -> None:
        self.db: Database = db

    def get_all_names(self) -> list[str]:
        rows: list[dict[str, str | int]] = self.db.query("SELECT * FROM users")
        return [str(row["name"]) for row in rows]


# Test using the stub
def test_get_all_names_returns_names() -> None:
    # Arrange
    stub_db: DatabaseStub = DatabaseStub()
    service: UserService = UserService(stub_db)

    # Act
    names: list[str] = service.get_all_names()

    # Assert
    assert names == ["Alice", "Bob"]
    print(f"Names from stub: {names}")
    print("Test PASSED: stub provided canned data without a real DB")


test_get_all_names_returns_names()

In [ ]:
# Configurable stubs let you test different scenarios
class ConfigurableDatabaseStub:
    """Stub with configurable responses."""

    def __init__(
        self,
        query_result: list[dict[str, str | int]] | None = None,
        should_raise: bool = False,
    ) -> None:
        self._query_result: list[dict[str, str | int]] = query_result or []
        self._should_raise: bool = should_raise

    def query(self, sql: str) -> list[dict[str, str | int]]:
        if self._should_raise:
            raise ConnectionError("Database unavailable")
        return self._query_result

    def execute(self, sql: str) -> int:
        if self._should_raise:
            raise ConnectionError("Database unavailable")
        return 1


# Scenario 1: Empty result
empty_stub: ConfigurableDatabaseStub = ConfigurableDatabaseStub(query_result=[])
service1: UserService = UserService(empty_stub)
result1: list[str] = service1.get_all_names()
assert result1 == []
print(f"Empty DB scenario: {result1}")

# Scenario 2: Single user
one_user_stub: ConfigurableDatabaseStub = ConfigurableDatabaseStub(
    query_result=[{"id": 1, "name": "Charlie"}]
)
service2: UserService = UserService(one_user_stub)
result2: list[str] = service2.get_all_names()
assert result2 == ["Charlie"]
print(f"One user scenario: {result2}")

# Scenario 3: Database error
error_stub: ConfigurableDatabaseStub = ConfigurableDatabaseStub(should_raise=True)
service3: UserService = UserService(error_stub)
try:
    service3.get_all_names()
except ConnectionError as e:
    print(f"Error scenario: caught {e}")

print("All stub scenarios PASSED")

## Section 4: Test Doubles -- Spies

A **spy** records how it was called while optionally delegating to a real implementation. Spies answer the question: *"Was my dependency called correctly?"*

In [ ]:
# A spy that records all calls
class LoggerSpy:
    """Records all log messages for later inspection."""

    def __init__(self) -> None:
        self.messages: list[tuple[str, str]] = []

    def log(self, level: str, message: str) -> None:
        self.messages.append((level, message))

    def info(self, message: str) -> None:
        self.log("INFO", message)

    def error(self, message: str) -> None:
        self.log("ERROR", message)


# Code under test
class DataImporter:
    """Imports data and logs the process."""

    def __init__(self, logger: LoggerSpy) -> None:
        self.logger: LoggerSpy = logger

    def import_data(self, records: list[str]) -> int:
        self.logger.info(f"Starting import of {len(records)} records")
        imported: int = 0
        for record in records:
            if record.strip():
                imported += 1
            else:
                self.logger.error("Skipped empty record")
        self.logger.info(f"Import complete: {imported} records imported")
        return imported


# Test using the spy
def test_import_logs_correctly() -> None:
    # Arrange
    spy: LoggerSpy = LoggerSpy()
    importer: DataImporter = DataImporter(spy)

    # Act
    count: int = importer.import_data(["alice", "", "bob"])

    # Assert on behavior (what was logged)
    assert count == 2
    assert len(spy.messages) == 3
    assert spy.messages[0] == ("INFO", "Starting import of 3 records")
    assert spy.messages[1] == ("ERROR", "Skipped empty record")
    assert spy.messages[2] == ("INFO", "Import complete: 2 records imported")

    print("Spy recorded messages:")
    for level, msg in spy.messages:
        print(f"  [{level}] {msg}")
    print(f"\nImported count: {count}")
    print("Test PASSED: spy verified correct logging behavior")


test_import_logs_correctly()

In [ ]:
# Spy that also delegates to a real implementation
class NotificationSpy:
    """Spy that wraps a real notifier and records calls."""

    def __init__(self) -> None:
        self.sent_notifications: list[dict[str, str]] = []

    def send(self, recipient: str, message: str) -> bool:
        self.sent_notifications.append(
            {"recipient": recipient, "message": message}
        )
        return True  # Simulate successful send

    @property
    def call_count(self) -> int:
        return len(self.sent_notifications)

    def was_sent_to(self, recipient: str) -> bool:
        return any(n["recipient"] == recipient for n in self.sent_notifications)


# Use the spy
spy: NotificationSpy = NotificationSpy()
spy.send("alice@test.com", "Welcome!")
spy.send("bob@test.com", "Your order shipped")

print(f"Total notifications sent: {spy.call_count}")
print(f"Sent to alice: {spy.was_sent_to('alice@test.com')}")
print(f"Sent to charlie: {spy.was_sent_to('charlie@test.com')}")
print(f"All notifications: {spy.sent_notifications}")

## Section 5: Test Doubles -- Fakes

A **fake** is a working implementation of a dependency that is simpler than the real one. Unlike stubs (which return canned data), fakes have actual logic. Common examples include in-memory databases and file system abstractions.

In [ ]:
from typing import Protocol


# Protocol defining the storage interface
class KeyValueStore(Protocol):
    def get(self, key: str) -> str | None: ...
    def set(self, key: str, value: str) -> None: ...
    def delete(self, key: str) -> bool: ...
    def keys(self) -> list[str]: ...


# Fake implementation: in-memory store (instead of Redis, for example)
class InMemoryKeyValueStore:
    """Fake key-value store backed by a dictionary."""

    def __init__(self) -> None:
        self._data: dict[str, str] = {}

    def get(self, key: str) -> str | None:
        return self._data.get(key)

    def set(self, key: str, value: str) -> None:
        self._data[key] = value

    def delete(self, key: str) -> bool:
        if key in self._data:
            del self._data[key]
            return True
        return False

    def keys(self) -> list[str]:
        return list(self._data.keys())


# Code under test
class CacheService:
    """A service that uses a key-value store for caching."""

    def __init__(self, store: KeyValueStore) -> None:
        self.store: KeyValueStore = store

    def get_or_compute(self, key: str, compute_fn: object) -> str:
        """Get from cache, or compute and store."""
        cached: str | None = self.store.get(key)
        if cached is not None:
            return cached
        value: str = compute_fn()  # type: ignore[operator]
        self.store.set(key, value)
        return value


# Test with fake
def test_cache_stores_computed_value() -> None:
    # Arrange
    fake_store: InMemoryKeyValueStore = InMemoryKeyValueStore()
    cache: CacheService = CacheService(fake_store)
    compute_count: list[int] = [0]

    def expensive_computation() -> str:
        compute_count[0] += 1
        return "computed_result"

    # Act: first call computes
    result1: str = cache.get_or_compute("key1", expensive_computation)
    # Act: second call uses cache
    result2: str = cache.get_or_compute("key1", expensive_computation)

    # Assert
    assert result1 == "computed_result"
    assert result2 == "computed_result"
    assert compute_count[0] == 1  # Only computed once
    assert fake_store.get("key1") == "computed_result"

    print(f"First call result: {result1}")
    print(f"Second call result: {result2}")
    print(f"Computation count: {compute_count[0]}")
    print(f"Stored keys: {fake_store.keys()}")
    print("Test PASSED: fake store enabled real caching behavior test")


test_cache_stores_computed_value()

## Section 6: Comparing Test Doubles

Each type of test double serves a different purpose. Choosing the right one depends on what you are testing.

In [ ]:
# Summary of test double types with examples

test_doubles: list[dict[str, str]] = [
    {
        "type": "Stub",
        "purpose": "Provide canned responses",
        "when_to_use": "You need to control what a dependency returns",
        "verifies": "Output/state of code under test",
        "example": "DatabaseStub that always returns [{id: 1, name: Alice}]",
    },
    {
        "type": "Spy",
        "purpose": "Record interactions",
        "when_to_use": "You need to verify how a dependency was called",
        "verifies": "Interactions with the dependency",
        "example": "LoggerSpy that records all log messages",
    },
    {
        "type": "Fake",
        "purpose": "Simplified working implementation",
        "when_to_use": "You need realistic behavior without external resources",
        "verifies": "Integration behavior with a simpler dependency",
        "example": "InMemoryKeyValueStore instead of Redis",
    },
    {
        "type": "Mock (unittest.mock)",
        "purpose": "Configurable stub + spy in one",
        "when_to_use": "You want maximum flexibility without writing a class",
        "verifies": "Both outputs and interactions",
        "example": "Mock(return_value=42) with assert_called_once_with()",
    },
]

for td in test_doubles:
    print(f"{td['type']}:")
    print(f"  Purpose:     {td['purpose']}")
    print(f"  When to use: {td['when_to_use']}")
    print(f"  Verifies:    {td['verifies']}")
    print(f"  Example:     {td['example']}")
    print()

## Section 7: Builder Pattern for Test Data

The **builder pattern** provides a fluent interface for constructing complex test objects. Instead of writing long constructors with many parameters, builders let you specify only the values that matter for each test.

In [ ]:
from __future__ import annotations


class UserBuilder:
    """Builder for creating test user dictionaries."""

    def __init__(self) -> None:
        self._name: str = "default_user"
        self._age: int = 25
        self._email: str = "default@test.com"
        self._role: str = "user"
        self._active: bool = True

    def with_name(self, name: str) -> UserBuilder:
        self._name = name
        return self

    def with_age(self, age: int) -> UserBuilder:
        self._age = age
        return self

    def with_email(self, email: str) -> UserBuilder:
        self._email = email
        return self

    def with_role(self, role: str) -> UserBuilder:
        self._role = role
        return self

    def inactive(self) -> UserBuilder:
        self._active = False
        return self

    def build(self) -> dict[str, str | int | bool]:
        return {
            "name": self._name,
            "age": self._age,
            "email": self._email,
            "role": self._role,
            "active": self._active,
        }


# Using the builder: only specify what matters for each test
default_user = UserBuilder().build()
print(f"Default user: {default_user}")

admin = UserBuilder().with_name("Alice").with_role("admin").build()
print(f"Admin user:   {admin}")

inactive = UserBuilder().with_name("Bob").with_age(30).inactive().build()
print(f"Inactive user: {inactive}")

# Each test only specifies the fields relevant to its assertion
assert admin["role"] == "admin"
assert inactive["active"] is False
print("\nBuilder pattern tests PASSED")

In [ ]:
from __future__ import annotations


# A more complex builder for an order object
class OrderBuilder:
    """Builder for creating test order objects."""

    def __init__(self) -> None:
        self._customer: str = "Test Customer"
        self._items: list[dict[str, str | float | int]] = []
        self._discount: float = 0.0
        self._shipping: str = "standard"

    def for_customer(self, name: str) -> OrderBuilder:
        self._customer = name
        return self

    def with_item(
        self, name: str, price: float, quantity: int = 1
    ) -> OrderBuilder:
        self._items.append({"name": name, "price": price, "quantity": quantity})
        return self

    def with_discount(self, percent: float) -> OrderBuilder:
        self._discount = percent
        return self

    def with_express_shipping(self) -> OrderBuilder:
        self._shipping = "express"
        return self

    def build(self) -> dict[str, object]:
        subtotal: float = sum(
            item["price"] * item["quantity"]  # type: ignore[operator]
            for item in self._items
        )
        total: float = subtotal * (1 - self._discount / 100)
        return {
            "customer": self._customer,
            "items": self._items,
            "subtotal": round(subtotal, 2),
            "discount": self._discount,
            "total": round(total, 2),
            "shipping": self._shipping,
        }


# Test: discount is applied correctly
def test_order_discount() -> None:
    # Arrange (builder makes intent clear)
    order = (
        OrderBuilder()
        .with_item("Widget", price=100.0)
        .with_discount(10)
        .build()
    )

    # Assert
    assert order["subtotal"] == 100.0
    assert order["total"] == 90.0
    print(f"Order with 10% discount: {order}")
    print("Discount test PASSED")


# Test: express shipping is set
def test_express_shipping() -> None:
    order = (
        OrderBuilder()
        .for_customer("Alice")
        .with_item("Gadget", price=50.0, quantity=2)
        .with_express_shipping()
        .build()
    )

    assert order["shipping"] == "express"
    assert order["total"] == 100.0
    print(f"\nExpress order: {order}")
    print("Express shipping test PASSED")


test_order_discount()
test_express_shipping()

## Section 8: Test Organization and Naming

Well-organized tests are easier to maintain and understand. Good naming conventions, logical grouping, and helper functions reduce duplication.

In [ ]:
# Test naming conventions
#
# Good names describe: WHAT is being tested and WHAT is expected
#
# Pattern: test_<unit>_<scenario>_<expected>
#   test_divide_by_zero_raises_value_error
#   test_sort_empty_list_returns_empty
#   test_login_invalid_password_returns_false
#
# Or: test_<action>_<condition> (simpler)
#   test_add_item_increases_count
#   test_remove_missing_item_raises

naming_examples: list[dict[str, str]] = [
    {"bad": "test_1", "good": "test_add_positive_numbers_returns_sum"},
    {"bad": "test_error", "good": "test_divide_by_zero_raises_value_error"},
    {"bad": "test_it_works", "good": "test_login_valid_credentials_returns_token"},
    {"bad": "test_edge", "good": "test_sort_empty_list_returns_empty_list"},
]

print("Test Naming Examples:")
print(f"{'Bad Name':<20} {'Good Name'}")
print("-" * 65)
for ex in naming_examples:
    print(f"{ex['bad']:<20} {ex['good']}")

In [ ]:
# Test grouping with classes (how pytest organizes tests)
#
# class TestShoppingCart:           # Group by unit
#     class TestAddItem:            # Group by method
#         def test_increases_count
#         def test_updates_total
#     class TestRemoveItem:
#         def test_decreases_count
#         def test_missing_item_raises

# Simulating grouped tests
class TestStack:
    """Tests for a stack data structure."""

    @staticmethod
    def _make_stack(items: list[int] | None = None) -> list[int]:
        """Helper to create a stack (reduces duplication)."""
        return list(items) if items else []

    def test_push_adds_to_top(self) -> None:
        stack: list[int] = self._make_stack()
        stack.append(42)
        assert stack[-1] == 42
        print("test_push_adds_to_top: PASSED")

    def test_pop_removes_from_top(self) -> None:
        stack: list[int] = self._make_stack([1, 2, 3])
        top: int = stack.pop()
        assert top == 3
        assert len(stack) == 2
        print("test_pop_removes_from_top: PASSED")

    def test_pop_empty_raises(self) -> None:
        stack: list[int] = self._make_stack()
        try:
            stack.pop()
            assert False, "Should have raised"
        except IndexError:
            print("test_pop_empty_raises: PASSED")

    def test_peek_does_not_remove(self) -> None:
        stack: list[int] = self._make_stack([10, 20])
        top: int = stack[-1]
        assert top == 20
        assert len(stack) == 2
        print("test_peek_does_not_remove: PASSED")


# Run grouped tests
suite: TestStack = TestStack()
print("TestStack:")
suite.test_push_adds_to_top()
suite.test_pop_removes_from_top()
suite.test_pop_empty_raises()
suite.test_peek_does_not_remove()

In [ ]:
# Shared fixtures and helper functions reduce duplication
#
# In pytest, you'd put shared fixtures in conftest.py:
#
# conftest.py:
#   @pytest.fixture
#   def sample_users() -> list[dict]:
#       return [
#           UserBuilder().with_name("Alice").build(),
#           UserBuilder().with_name("Bob").with_role("admin").build(),
#       ]

# Simulating shared test infrastructure
def make_test_users(count: int = 3) -> list[dict[str, str | int | bool]]:
    """Factory function for creating test users."""
    names: list[str] = ["Alice", "Bob", "Charlie", "Diana", "Eve"]
    return [
        UserBuilder()
        .with_name(names[i])
        .with_age(25 + i * 5)
        .with_email(f"{names[i].lower()}@test.com")
        .build()
        for i in range(min(count, len(names)))
    ]


# Tests use the factory
users: list[dict[str, str | int | bool]] = make_test_users(3)
print("Test users from factory:")
for user in users:
    print(f"  {user['name']} (age={user['age']}, email={user['email']})")

assert len(users) == 3
assert users[0]["name"] == "Alice"
assert users[2]["name"] == "Charlie"
print("\nFactory function test PASSED")

## Section 9: Putting It All Together

A complete example combining AAA pattern, test doubles, and builder pattern to test a realistic system.

In [ ]:
from __future__ import annotations


# --- Production Code ---

class NotificationService:
    """Sends notifications to users based on their preferences."""

    def __init__(self, email_sender: object, sms_sender: object) -> None:
        self.email_sender = email_sender
        self.sms_sender = sms_sender

    def notify(
        self, user: dict[str, object], message: str
    ) -> list[str]:
        """Send notification via user's preferred channels."""
        channels_used: list[str] = []
        prefs = user.get("preferences", {})

        if prefs.get("email"):  # type: ignore[union-attr]
            self.email_sender.send(  # type: ignore[attr-defined]
                to=user["email"], body=message
            )
            channels_used.append("email")

        if prefs.get("sms"):  # type: ignore[union-attr]
            self.sms_sender.send(  # type: ignore[attr-defined]
                to=user["phone"], body=message
            )
            channels_used.append("sms")

        return channels_used


# --- Test Infrastructure ---

class SenderSpy:
    """Spy for email/SMS senders."""

    def __init__(self) -> None:
        self.sent: list[dict[str, str]] = []

    def send(self, to: str, body: str) -> None:
        self.sent.append({"to": to, "body": body})


class NotificationUserBuilder:
    """Builder for users with notification preferences."""

    def __init__(self) -> None:
        self._name: str = "Test User"
        self._email: str = "test@example.com"
        self._phone: str = "+1234567890"
        self._pref_email: bool = False
        self._pref_sms: bool = False

    def with_name(self, name: str) -> NotificationUserBuilder:
        self._name = name
        return self

    def with_email_notifications(self) -> NotificationUserBuilder:
        self._pref_email = True
        return self

    def with_sms_notifications(self) -> NotificationUserBuilder:
        self._pref_sms = True
        return self

    def build(self) -> dict[str, object]:
        return {
            "name": self._name,
            "email": self._email,
            "phone": self._phone,
            "preferences": {
                "email": self._pref_email,
                "sms": self._pref_sms,
            },
        }


# --- Tests ---

def test_email_only_user_receives_email() -> None:
    # Arrange
    email_spy: SenderSpy = SenderSpy()
    sms_spy: SenderSpy = SenderSpy()
    service: NotificationService = NotificationService(email_spy, sms_spy)
    user = NotificationUserBuilder().with_name("Alice").with_email_notifications().build()

    # Act
    channels: list[str] = service.notify(user, "Hello!")

    # Assert
    assert channels == ["email"]
    assert len(email_spy.sent) == 1
    assert email_spy.sent[0]["body"] == "Hello!"
    assert len(sms_spy.sent) == 0
    print("test_email_only: PASSED")


def test_both_channels_user_receives_both() -> None:
    # Arrange
    email_spy: SenderSpy = SenderSpy()
    sms_spy: SenderSpy = SenderSpy()
    service: NotificationService = NotificationService(email_spy, sms_spy)
    user = (
        NotificationUserBuilder()
        .with_name("Bob")
        .with_email_notifications()
        .with_sms_notifications()
        .build()
    )

    # Act
    channels: list[str] = service.notify(user, "Alert!")

    # Assert
    assert channels == ["email", "sms"]
    assert len(email_spy.sent) == 1
    assert len(sms_spy.sent) == 1
    print("test_both_channels: PASSED")


def test_no_preferences_sends_nothing() -> None:
    # Arrange
    email_spy: SenderSpy = SenderSpy()
    sms_spy: SenderSpy = SenderSpy()
    service: NotificationService = NotificationService(email_spy, sms_spy)
    user = NotificationUserBuilder().with_name("Charlie").build()

    # Act
    channels: list[str] = service.notify(user, "Nothing")

    # Assert
    assert channels == []
    assert len(email_spy.sent) == 0
    assert len(sms_spy.sent) == 0
    print("test_no_preferences: PASSED")


print("Notification Service Tests:")
test_email_only_user_receives_email()
test_both_channels_user_receives_both()
test_no_preferences_sends_nothing()
print("\nAll tests PASSED")

## Summary

### AAA Pattern (Arrange-Act-Assert)
- **Arrange**: Set up test data, create objects, configure dependencies
- **Act**: Execute the single behavior being tested
- **Assert**: Verify the expected outcome (return value, state change, or interaction)
- Keep each test focused on one behavior; avoid multiple Act phases

### Test Doubles
- **Stub**: Returns canned data; use when you need to control what a dependency returns
- **Spy**: Records interactions; use when you need to verify how a dependency was called
- **Fake**: Simplified working implementation; use when you need realistic behavior without external resources
- **Mock** (`unittest.mock`): Combines stub + spy functionality in a single configurable object

### Builder Pattern
- Creates complex test objects with a fluent interface (`builder.with_x().with_y().build()`)
- Provides sensible defaults so tests only specify values relevant to the assertion
- Reduces noise and makes test intent clear

### Test Organization
- **Naming**: `test_<unit>_<scenario>_<expected>` makes tests self-documenting
- **Grouping**: Use test classes to group related tests by unit or feature
- **Helpers**: Factory functions and fixture functions reduce duplication
- **Shared fixtures**: Place in `conftest.py` for cross-module reuse
- Test through public interfaces, not internal state